# Tools for Big Data: Web scraping

In [1]:
#Import libraries
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import pandas as pd
import re

In [11]:
book_data = []

url1 = [
    'https://books.toscrape.com/catalogue/category/books/health_47/index.html',
    'https://books.toscrape.com/catalogue/category/books/science-fiction_16/index.html',
    'https://books.toscrape.com/catalogue/category/books/humor_30/index.html',
    'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
    'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html'
]
#Webscraper for the books 
for url in url1:
    response1 = requests.get(url)
    soup1 = BeautifulSoup(response1.text, 'html.parser')
    books = soup1.find_all("article", class_="product_pod")

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text[2:]
        rating = book.p["class"][1]
        stock = book.find("p", class_="instock availability").get_text().strip()
        category = url.split('/')[-2]
        image = url.replace("index.html", "") + book.find("img").get("src")

        # Extracting the URL for each book
        book_url = url.replace("index.html", "") + book.h3.a["href"]

        # Requesting the individual book page
        response_book = requests.get(book_url)
        soup_book = BeautifulSoup(response_book.text, 'html.parser')

        # Webscraping the description 
        description = soup_book.find_all('article', attrs={"class": "product_page"})[0].find_all('p')[3].get_text(strip=True)

        book_data.append({
            'Title': title,
            'Price': price,
            'Rating': rating,
            'Availability': stock,
            'Description': description,
            'Category': category,
            'Image Url': image
        })

# Creating a DataFrame from the collected data
books_df = pd.DataFrame(book_data)

#Saving the dataframe to a CSV file
books_df.to_csv('books_data.csv', index= False)
